In [1]:
# ResNet
# 加深神经网络一定有好处吗

In [1]:
# 需要保证无偏，就是对之前模型的严格包含
# 保证之后的模型一定不会比之前的模型差
# 在函数空间上后面包含前面

In [2]:
# 残差快
# 串联一个层改变函数类，我们希望扩大函数类
# 残差快加入快速通道(右边来得到)
# f(x)=x+g(x)的结构
# 这样函数空间至少不必原来的差
# f_{i+1}>f_{i} (函数空间的理解)
# 防止网络退化
# VGG块的变种

In [3]:
# ResNet 架构
# --类似VGG和GoogleNet的架构
# 但替换成了ResNet块
# ResNet 块 两种(宽高不变，宽高减半)
# 刷分 Res152
# 工程 Res50,Res34,Res18
# ResNet 变种(连接思想太重要了)

In [4]:
# 残差块 使得很深的网络更加容易训练
# 甚至可以训练1k层的神经网络
# 残差网络对随后的深层神经网络设计，产生了深远的影响
# 卷积层 & 全连接层

In [5]:
import torch 
from torch import nn 
from torch.nn import functional as F 
from d2l import torch as d2l 

In [8]:
class Residual(nn.Module):
    def __init__(self,input_channels,num_channels,
                 use_1x1conv=False,strides=1):
        super().__init__()
        self.conv1=nn.Conv2d(input_channels,num_channels,kernel_size=3,stride=strides,padding=1)
        self.conv2=nn.Conv2d(num_channels,num_channels,kernel_size=3,padding=1)
        if use_1x1conv:
            self.conv3=nn.Conv2d(input_channels,num_channels,kernel_size=1,stride=strides)
        else: 
            self.conv3=None 
        self.bn1=nn.BatchNorm2d(num_channels)
        self.bn2=nn.BatchNorm2d(num_channels)
        
    def forward(self,X):
        Y=F.relu(self.bn1(self.conv1(X)))
        Y=self.bn2(self.conv2(Y))
        if self.conv3:
            X=self.conv3(X)
        Y+=X 
        return F.relu(Y)

In [9]:
blk = Residual(3, 3)
X = torch.rand(4, 3, 6, 6)
Y = blk(X)
Y.shape


torch.Size([4, 3, 6, 6])

In [10]:
blk = Residual(3, 6, use_1x1conv=True, strides=2)
blk(X).shape


torch.Size([4, 6, 3, 3])

In [ ]:
b1=nn.Sequential(
    nn.Conv2d(1,64,kernel_size=7,stride=2,padding=3),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
)

def resnet_block(
    input_channels,num_channels,num_residuals,first_block=False
):
    blk=[]
    for i in range(num_residuals):
        if i==0:
            if first_block:
                blk.append(
                    Residual(input_channels,num_channels,use_1x1conv=True)
                )
            else: 
                